# Проект A/B-тестирование

## Постановка задачи

Ваша задача — провести оценку результатов A/B-теста. В вашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.

- Оцените корректность проведения теста
- Проанализируйте результаты теста

Чтобы оценить корректность проведения теста, проверьте:

- пересечение тестовой аудитории с конкурирующим тестом,
- совпадение теста и маркетинговых событий, другие проблемы временных границ теста.

## Техническое задание

- Название теста: `recommender_system_test`;
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.

### Данные

[final_ab_events.csv](https://disk.yandex.ru/d/pgKgZ9lRp0Enwg)<br> https://code.s3.yandex.net/datasets/final_ab_events.csv

[ab_project_marketing_events.csv](https://disk.yandex.ru/d/ZVC7GUYYckKnGQ) <br> https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv

[final_ab_new_users.csv](https://disk.yandex.ru/d/Qrto8Gerpu424g) <br> https://code.s3.yandex.net/datasets/final_ab_new_users.csv

[final_ab_participants.csv](https://disk.yandex.ru/d/nhGUR3ZkNIG4VA) <br> https://code.s3.yandex.net/datasets/final_ab_participants.csv

### Описание данных

`ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год.

Структура файла:

- `name` — название маркетингового события;
- `regions` — регионы, в которых будет проводиться рекламная кампания;
- `start_dt` — дата начала кампании;
- `finish_dt` — дата завершения кампании.

`final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

Структура файла:

- `user_id` — идентификатор пользователя;
- `first_date` — дата регистрации;
- `region` — регион пользователя;
- `device` — устройство, с которого происходила регистрация.

`final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура файла:

- `user_id` — идентификатор пользователя;
- `event_dt` — дата и время покупки;
- `event_name` — тип события;
- `details` — дополнительные данные о событии. Например, для покупок, `purchase,` в этом поле хранится стоимость покупки в долларах.

`final_ab_participants.csv` — таблица участников тестов.

Структура файла:

- `user_id` — идентификатор пользователя;
- `ab_test` — название теста;
- `group` — группа пользователя

## Как сделать задание?

- Опишите цели исследования
- Исследуйте данные:
    - Требуется ли преобразование типов?
    - Опишите природу пропущенных значений и дубликатов, если их обнаружите.
- Оцените корректность проведения теста. Обратите внимание на:
    - Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.
    - Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.
    - Аудиторию теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения по тестовым группам и правильность их формирования.
- Проведите исследовательский анализ данных:
    - Количество событий на пользователя одинаково распределены в выборках?
    - Как число событий в выборках распределено по дням?
    - Как меняется конверсия в воронке в выборках на разных этапах?
    - Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?
- Оцените результаты A/B-тестирования
    - Что можно сказать про результаты A/В-тестирования?
    - Проверьте статистическую разницу долей z-критерием.
- Опишите выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования. Сделайте общее заключение о корректности проведения теста.

# Описание цели исследования

<b>Задача:</b> провести оценку A\B-тестирования новой рекомендательной системы товаров для пользователей. Оценить изменились ли продажи после внедрения изменений или нет.<br>

Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:<br>

конверсии в просмотр карточек товаров — событие product_page,<br>
просмотры корзины — product_cart,<br>
покупки — purchase.<br>

## Загрузка и подготовка данных


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import plotly.graph_objects as go
import plotly.express as px
import math as mth
from scipy import stats as st
import warnings
warnings.filterwarnings('ignore')

## Чтение таблиц



[final_ab_events.csv](https://disk.yandex.ru/d/pgKgZ9lRp0Enwg)<br> https://code.s3.yandex.net/datasets/final_ab_events.csv

[ab_project_marketing_events.csv](https://disk.yandex.ru/d/ZVC7GUYYckKnGQ) <br> https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv

[final_ab_new_users.csv](https://disk.yandex.ru/d/Qrto8Gerpu424g) <br> https://code.s3.yandex.net/datasets/final_ab_new_users.csv

[final_ab_participants.csv](https://disk.yandex.ru/d/nhGUR3ZkNIG4VA) <br> https://code.s3.yandex.net/datasets/final_ab_participants.csv

In [2]:
marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
#marketing_events = pd.read_csv('/Users/artemavdosin/Desktop/ABtest/ab_project_marketing_events.csv')
final_events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
#final_events = pd.read_csv('/Users/artemavdosin/Desktop/ABtest/final_ab_events.csv')
new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
#new_users = pd.read_csv('/Users/artemavdosin/Desktop/ABtest/final_ab_new_users.csv')
participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')
#participants = pd.read_csv('/Users/artemavdosin/Desktop/ABtest/final_ab_participants.csv')



<b>Календарь маркетинговых событий на 2020 год.</b>

In [3]:
marketing_events.head()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


Структура файла:<br>

name — название маркетингового события;<br>
regions — регионы, в которых будет проводиться рекламная кампания;<br>
start_dt — дата начала кампании;<br>
finish_dt — дата завершения кампании.<br>

<b>Действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.</b>

In [4]:
final_events.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


Структура файла:<br>
    
user_id — идентификатор пользователя;<br>
event_dt — дата и время покупки;<br>
event_name — тип события;<br>
details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.<br>

<b>Пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.</b>

In [5]:
new_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


Структура файла:<br>

user_id — идентификатор пользователя;<br>
first_date — дата регистрации;<br>
region — регион пользователя;<br>
device — устройство, с которого происходила регистрация.<br>

<b>Таблица участников тестов</b>

In [6]:
participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


Структура файла:<br>

user_id — идентификатор пользователя;<br>
ab_test — название теста;<br>
group — группа пользователя.<br>

## Предобработка данных


### Предобработка Календарь маркетинговых событий на 2020 год. 

### Исследуйте данные:

- Требуется ли преобразование типов?

- Опишите природу пропущенных значений и дубликатов, если их обнаружите.

In [7]:
marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


In [8]:
#Поиск дубликатов 
print(marketing_events.duplicated().sum())

0


- В таблице marketing_events нет пропусков и дубликтов.
- У столбцов со временем неправильный тип данных.

In [9]:
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
display(marketing_events.head())
print(' ')
display(marketing_events.info())

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


None

Датасет готов

### Предобработка Действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

In [10]:
final_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


In [11]:
#Поиск дубликатов 
print(final_events.duplicated().sum())

0


In [12]:
print(final_events.isnull().sum()*100/len(final_events))

user_id        0.000000
event_dt       0.000000
event_name     0.000000
details       85.751175
dtype: float64


- Имеются пропуски в details. Нужно посмотреть, соответствует ли это  ТЗ.
- Нужно исправить тип данных у столбца с датой.


In [13]:
final_events_nan = final_events.groupby('event_name')['details'].unique()
display(final_events_nan)

event_name
login                                 [nan]
product_cart                          [nan]
product_page                          [nan]
purchase        [99.99, 9.99, 4.99, 499.99]
Name: details, dtype: object

Данные в столбце с событиями: оплата, корзина с товаром, страница с товаром, вход. Пропуски в
данных находятся в деталях к событиям не подтверждающихся оплатой или иными дополнительными
фиксациями переход от события к событию и есть подтверждающая деталь технического задания. Удалить пропуски в details не стоит. 

In [14]:
final_events['event_dt'] = pd.to_datetime(final_events['event_dt'])

display(final_events.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     440317 non-null  object        
 1   event_dt    440317 non-null  datetime64[ns]
 2   event_name  440317 non-null  object        
 3   details     62740 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 13.4+ MB


None

Датасет готов

### Предобработка Пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

In [15]:
new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [16]:
print(new_users.duplicated().sum())

0


- Дубликатов и пропусков нет 
- Нужно изменить тип данных у столбца с датой 

In [17]:
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
display(new_users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


None

Датасет готов

Вывод
- Дубликатов в данных нет 
- Все даннае с датой приведены к корректному типу данных 
- Обнарузены пропуски в даннных деталях событий, это соответствует ТЗ и удалению не подлежат

- Пропустов и дубликатов нет. 
- Все типы данных верны

С данными можно работать

### Проверка технического задания 


- Оцените корректность проведения теста. Обратите внимание на:
    - Соответствие данных требованиям технического задания. Проверьте корректность всех пунктов технического задания.
    - Время проведения теста. Убедитесь, что оно не совпадает с маркетинговыми и другими активностями.
    - Аудиторию теста. Удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно. Проверьте равномерность распределения по тестовым группам и правильность их формирования.
    
    

- Название теста: `recommender_system_test`;
- группы: А — контрольная, B — новая платёжная воронка;
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;
- аудитория: 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.
- ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    - конверсии в просмотр карточек товаров — событие `product_page`,
    - просмотры корзины — `product_cart`,
    - покупки — `purchase`.


#### Название теста: recommender_system_test
группы: А — контрольная, B — новая платёжная воронка;

In [18]:
participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


In [19]:
participants.groupby(['ab_test', 'group']).agg({'ab_test': 'count'})

ab_test
ab_test                 group         
interface_eu_test       A         5831
                        B         5736
recommender_system_test A         3824
                        B         2877

Видим, что в таблице участников теста, есть две тестовые группы interface_eu_test и recommender_system_test

In [20]:
print('Исследование наличия пользователей, на попадание в обе группы:')
display(participants.groupby('user_id').agg({'group':'nunique'}).reset_index().sort_values('group', ascending=False))

Исследование наличия пользователей, на попадание в обе группы:


,user_id,group
7402,72742C5F312A1FEC,2
11050,A929DCBED46E31D1,2
5068,4E971B47912497CE,2
7780,786EB537736D5055,2
5882,5B4EFE916AD19741,2
...,...,...
5680,57E796E81DD60883,1
5681,57E945ACB21E1DA8,1
5682,57EEF3B13ABF1FE2,1
5684,57FD266D6A09BCD2,1


In [21]:
print('Исследование наличия пользователей, на попадание в обе группы для необходимого теста:')
participants_test=participants[participants['ab_test']=='recommender_system_test']
display(participants_test.groupby('user_id').agg({'group':'nunique'}).reset_index().sort_values('group', ascending=False))


Исследование наличия пользователей, на попадание в обе группы для необходимого теста:


,user_id,group
0,000ABE35EE11412F,1
4502,AB3E2847DF7EC531,1
4474,AA703FB9F9004F86,1
4473,AA703605654827AE,1
4472,AA5A1803D3FA76B4,1
...,...,...
2231,56AEC8FDA4BCA549,1
2230,569CEF5F441978B3,1
2229,5698C00E2A472E8E,1
2228,568C35FDE36F7C26,1


In [22]:
print()
print('Количество пользователей в группах для двух тестов:')
display(participants.groupby('group')['user_id'].nunique())
display(participants)


Количество пользователей в группах для двух тестов:


group
A    9173
B    8269
Name: user_id, dtype: int64

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test
...,...,...,...
18263,1D302F8688B91781,B,interface_eu_test
18264,3DE51B726983B657,A,interface_eu_test
18265,F501F79D332BE86C,A,interface_eu_test
18266,63FBE257B05F2245,A,interface_eu_test


In [23]:
print('Уникальность пользователей в группах для двух тестов:')
display(participants.groupby('user_id')['group'].unique())
print()

Уникальность пользователей в группах для двух тестов:


user_id
0002CE61FF2C4011       [A]
000ABE35EE11412F       [A]
001064FEAAB631A1       [B]
0010A1C096941592       [A]
001C05E87D336C59       [A]
                     ...  
FFE858A7845F005E       [A]
FFED90241D04503F       [B]
FFEFC0E55C1CCD4F       [B]
FFF28D02B1EACBE1    [B, A]
FFF58BC33966EB51       [B]
Name: group, Length: 16666, dtype: object

In [24]:
print('Количество уникальных пользователей в тесте recommender_system_test в группах:')
participants.groupby('group')['user_id'].nunique()

Количество уникальных пользователей в тесте recommender_system_test в группах:


group
A    9173
B    8269
Name: user_id, dtype: int64

В датасете с изучаемым тестом нет уникальных пользователей попавших в обе группы теста, но
количество участников группы В на 1000 пользователей меньше.

In [25]:

test1 = participants.query('ab_test == "recommender_system_test"')
test2 = participants.query('ab_test == "interface_eu_test"')

ff = test1.merge(test2, on='user_id', how='inner')

unique_users = ff['user_id'].unique().tolist()
print('Количество пересечений между разными тестами:',len(unique_users))

Количество пересечений между разными тестами: 1602


#### Проверка дат запуска и остановки тестов
- дата запуска: 2020-12-07;
- дата остановки набора новых пользователей: 2020-12-21;
- дата остановки: 2021-01-04;

<b>Объединённый датасет с информацией о клиентах и их регистрации, и информацией о тесте и тестируемой
группе.</b>

In [26]:
data = new_users.merge(participants, on='user_id', how='left')
display(data.head())
data.info()

,user_id,first_date,region,device,group,ab_test
0,D72A72121175D8BE,2020-12-07,EU,PC,A,recommender_system_test
1,F1C668619DFE6E65,2020-12-07,N.America,Android,NaN,NaN
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC,A,interface_eu_test
3,50734A22C0C63768,2020-12-07,EU,iPhone,B,interface_eu_test
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 63335 entries, 0 to 63334
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     63335 non-null  object        
 1   first_date  63335 non-null  datetime64[ns]
 2   region      63335 non-null  object        
 3   device      63335 non-null  object        
 4   group       18268 non-null  object        
 5   ab_test     18268 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 3.4+ MB


В датасете много пропущенных данных о виде теста и нахождения пользователя в группе, не все
пользователи были разделены или данные пользователи были зарегистрированы ранее.

<b>Объединённый датасет с информацией о событии совершенном пользователем</b>

In [27]:
data = data.merge(final_events, on='user_id', how='left')

In [28]:
display(data.tail(10))
data.info()

,user_id,first_date,region,device,group,ab_test,event_dt,event_name,details
449946,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-23 15:47:23,product_page,NaN
449947,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-25 12:31:05,product_page,NaN
449948,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-27 03:51:36,product_page,NaN
449949,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-20 14:14:28,login,NaN
449950,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-21 04:31:15,login,NaN
449951,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-23 15:47:23,login,NaN
449952,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-25 12:31:04,login,NaN
449953,1C7D23927835213F,2020-12-20,EU,iPhone,B,interface_eu_test,2020-12-27 03:51:35,login,NaN
449954,8F04273BB2860229,2020-12-20,EU,Android,NaN,NaN,2020-12-20 03:17:17,product_cart,NaN
449955,8F04273BB2860229,2020-12-20,EU,Android,NaN,NaN,2020-12-20 03:17:17,login,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 449956 entries, 0 to 449955
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     449956 non-null  object        
 1   first_date  449956 non-null  datetime64[ns]
 2   region      449956 non-null  object        
 3   device      449956 non-null  object        
 4   group       110368 non-null  object        
 5   ab_test     110368 non-null  object        
 6   event_dt    446211 non-null  datetime64[ns]
 7   event_name  446211 non-null  object        
 8   details     63588 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 34.3+ MB


#### Проверка на дату завершения набора новых пользователей нашего теста

 По ТЗ дата остановки набора новых пользователей: 2020-12-21. Нам следует отфильтровать события с лайфтаймом больше, чем горизонт событий. (т.е. удалить строки, в которых события больше 14 дней)

In [29]:
data['event_dt'] = pd.to_datetime(data['event_dt'])
data['days'] = data['event_dt']-data['first_date']
data=data.query('days<="14 days" or days=="NaT"')
del data['days']



ValueError: unknown type timedelta64[ns]

<b>Добавление данных из таблицы с маркетинговыми событиями:</b>

In [ ]:
data['first_date_month']=data['first_date'].astype('datetime64[M]')
marketing_events['start_dt_month']=marketing_events['start_dt'].astype('datetime64[M]')
marketing_events


In [ ]:
# посмотрим на маркетинговые события в период АБ теста
marketing_events = marketing_events.query('start_dt >= "2020-12-07" and finish_dt <= "2021-01-04"')
print('События маркетинговых активностей в период теста')
marketing_events

In [ ]:
data = data.merge(marketing_events, left_on='first_date_month', right_on='start_dt_month')

In [ ]:
display(data.head())

<b>Вывод по общему датафрейму:</b>
- Со временем проведения теста совпадают две маркетинговые кампании проводящиеся на разные регионы, на
СНГ и Европу и Северную Америку. 
- Камапания направленная на EU началась 25 декабря и закончилась 03 января. 
- Конечный период совпадает с окончанием теста, но не совпадает с началом. 
- Данная акция даже поназванию, скорее направлена на поддержание спроса в период с рождества в Европе с 25 декабря до нового
года, из-за иссякающего у европейцев спроса после рождественских покупок.


#### Ожидаемое количество участников теста: 6000

<b>Доля пользователей попавших в группы А и В:</b>

In [ ]:
# групировка для пользователей попавших в обе группы
participants_group= participants.groupby(['user_id']).agg({'group': 'nunique'}).reset_index().sort_values('group', ascending=False)

# общее количество пользователей попавших в тесты:
print('Oбщее количество пользователей попавших в тесты')
participants_users = participants_group['user_id'].count()
print(participants_users)
print()

Это соответствует техническому заданию

In [ ]:
# количество пользователей попавших в две группы:
participants_a_b = participants_group[participants_group['group'] > 1]['user_id'].count()
print('Oбщее количество пользователей попавших в обе группы')
print(participants_a_b)
print()

In [ ]:
participants_a_b_2 = participants.groupby('group')['user_id'].nunique().sum()-participants['user_id'].nunique()

In [ ]:
print('Oбщее количество пользователей попавших в обе группы')
print(participants_a_b_2)

In [ ]:
# доля пользователей попавших в обе группы
print('Доля пользователей попавших в обе группы')
participants_ratio = round(participants_a_b_2/participants_users*100, 2)
print(participants_ratio, '%')
print()

- Доля пользователей попавших в обе группы относительно относительно всех пользователей крайне мала и так
как группы при этом находятся в разных тестах, данное разделение не отразится на результатах теста.

#### Аудитория: 
- 15% новых пользователей из региона EU;
- назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- ожидаемое количество участников теста: 6000.

In [ ]:
data.head()

In [ ]:
new_users.head()

In [ ]:
print('Аудитория пользователей из региоа EU и тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;')
sample = data.query('ab_test=="recommender_system_test" and region== "EU" and  start_dt >= "2020-12-07" and finish_dt <= "2021-01-04"')['user_id'].nunique()
population = new_users.query('region== "EU" and first_date > "2020-12-07"')['user_id'].nunique()
print(round(sample/population, 2))



Это соответствует нашему техническому заданию


In [ ]:
users_eu = data[data['region']=='EU']['user_id'].nunique()
print('Количество пользователей из EU')
print(population)
print()
print('Количество пользователей из EU попавших в тест')
print(sample)
print()
# доля пользователей попавших в тест
print('Доля пользователей EU попавших в тест')
print(round(sample/population, 3), '%')
print()
# доля пользователей попавших в тест
print('Количество пользователей EU ожидаемых в тесте:')
users_eu_wait = population*0.15
print(users_eu_wait)
print()

Количество пользователей больше чем 6000. Это соответствует ТЗ 

In [ ]:
#Проверка на дату завершения набора новых пользователей нашего теста
# В тесте не должны участвовать пользователи с датой регистрации после 2020-12-21 
data = data.query('first_date<="2020-12-21"')

#### Общий вывод по проверке технического задания 


В данных было обнаружено два теста 
interface_eu_test и recommender_system_test - нас интересует рекомендательная система

Количество пересечений между разными тестами recommender_system_test и interface_eu_test: 1602

Были отфильтрованы события с лайфтаймом больше, чем горизонт событий 14 дней 
Со временем проведения теста совпадают две маркетинговые кампании проводились на разные регионы, на СНГ и Европу и Северную Америку.
Кампания направленная на EU началась 25 декабря и закончилась 03 января.
Конечный период совпадает с окончанием теста, но не совпадает с началом.
Данная акция даже по названию, скорее направлена на поддержание спроса в период с рождества в Европе с 25 декабря до нового года, из-за неиссякающего у европейцев спроса после рождественских покупок.

Общее количество пользователей попавших в тесты 16666
Общее количество пользователей попавших в обе группы 776,  4.66 % от общей группы 

Доля пользователей EU попавших в тест  15 %

Количество пользователей EU ожидаемых в тесте: 6309.75 – это соответствует ТЗ (6000 пользователей)


В тесте были отфильтрованы пользователи которые зарегистрировались  после 2020-12-21 

### Исследовательский анализ данных

In [ ]:
data = data[data['ab_test']=='recommender_system_test']

In [ ]:
data.groupby('user_id').agg({'group':'nunique'}).reset_index().sort_values('group', ascending=False)
print('Количество пользователей в группах:')
data.groupby('group')['user_id'].nunique()


In [ ]:
print('Сводная таблица с количеством событий в логе:')
event_count = data.groupby('event_name').agg({'user_id':'count'}).sort_values(by='user_id',  ascending=False)
display(event_count)

In [ ]:
# Исследуем воронки на разных этапах  Посмотрим количество каждого события
ev_group = data.groupby(['group','event_name']).agg(
    {'user_id':'count'}).reset_index()
ev_group

In [ ]:
labels_ev = {'event_name':'Название событий', 'group':'Группа', 'user_id':'Количетсво событий'}
px.bar(ev_group, x = 'group',
         y = 'user_id', text = 'user_id',color = 'event_name', labels = labels_ev,
        title = 'Количетсво события по частоте в каждой группе', barmode = 'group')

In [ ]:
print("Всего событий А = {}".format(ev_group[ev_group['group']=='A']['user_id'].sum()))
print("Всего событий B = {}".format(ev_group[ev_group['group']=='B']['user_id'].sum()))
# Посмотрим скоолько пользователей совершили каждое из событий
us_group = data.groupby(['group','event_name']).agg({'user_id':'nunique'}).sort_values(
    by = 'user_id', ascending = False).reset_index()


In [ ]:
px.bar(
    us_group, x = 'group', y = 'user_id', text = 'user_id',color = 'event_name', barmode = 'group',
    labels = labels_ev,title = 'Количество пользователей, совершивших каждое событие, по группам')



In [ ]:
print("Всего пользователей A  = {}".format(us_group[us_group['group'] == 'A']['user_id'].sum()))
print("Всего пользователей B = {}".format(us_group[us_group['group'] == 'B']['user_id'].sum()))

In [ ]:
fun_a = data.query('group == "A"')\
                .groupby('event_name')['user_id'].nunique().reset_index().sort_values('user_id', ascending = False).reindex(index=[0, 2, 1, 3])
fun_b = data.query('group == "B"')\
                .groupby('event_name')['user_id'].nunique().reset_index().sort_values('user_id', ascending = False).reindex(index=[0, 2, 1, 3])

In [ ]:
fig = go.Figure()


fig.add_trace(go.Funnel(
    name = 'A',
    y = fun_a['event_name'],
    x = fun_a['user_id'],
    textinfo = "value+percent previous"))

fig.add_trace(go.Funnel(
    name = 'B',
    y = fun_b['event_name'],
    x = fun_b['user_id'],
    textinfo = "value+percent previous"))

fig.update_layout(
    title = {
        'text' : 'Конверсия событий пользователей по группам А и В)',
        'x':0.5
    }
)

fig.show()

Вывод <br>
Группа А
- В группе А  18801 событий, 6223 пользователей
- В воронке: Переход с логина на просмотр карточки товара 65%
Переход с карточки товара в корзину 46%
Переход из корзины в покупку в корзину 106% (пользователей больше 100%, это может быть связано с переходом к покупке сразу с карточки товара)

Группа B
- В группе B  5055 событий, 1961 пользователей
- В воронке: Переход с логина на просмотр карточки товара 56%
Переход с карточки товара в корзину 49%
Переход из корзины в покупку в корзину 100% 




In [ ]:
# Посмторим среднее количество событий на одного пользователя по дням
data['day'] = data['event_dt'].dt.day

mean_user_group = data.groupby(['group','user_id', 'day']).agg({'event_name':'count'}).reset_index()
print('В среднем на одного пользователя группа А приходится {:.2f} событий в день'.format(
    mean_user_group[mean_user_group['group'] == 'A']['event_name'].mean()))
print('В среднем на одного пользователя группы В приходится {:.2f} событий в день'.format(
    mean_user_group[mean_user_group['group'] == 'B']['event_name'].mean()))
labels_mug = {'event_name':'Количество событий на пользователя',
              'count':'Количетсво пользователей', 'group':'Группа'}
px.histogram(mean_user_group, x = 'group', color = 'event_name',
               labels = labels_mug, barmode = 'group',
               title = 'Распределение количества событий по группам, по количеству пользователей')


In [ ]:
# посмотрим на маркетинговые события в период АБ теста
marketing_events = marketing_events.query('start_dt >= "2020-12-07" and finish_dt <= "2021-01-04"')
print('События маркетинговых активностей в период теста')
marketing_events


In [ ]:
# Посмотрим на распределение событий по дням.
count_day_ev = data.groupby(['group','day']).agg({'event_name':'count'}).reset_index()
labels_cde = {'event_name':'Количество событий', 'day':'День месяца'}
px.histogram(count_day_ev, x = 'day', y = 'event_name',
               range_x = [5, 31],nbins = 20 ,color = 'group',
               labels = labels_cde, barmode = 'overlay',color_discrete_sequence=['green','red'],
              title = "Распределние количества событий по дням")


<div class="alert alert-block alert-warning">📝
    

__Комментарий от тимлида №3__


Как думаешь, с чем может быть связан пик событий в группе А 21 числа?
</div>

<div class="alert alert-info"> <b>Думаю, что это последние выходные перед европейским рождеством, все бегут покупать подарки </div>

In [ ]:
count_day_ev = data.groupby(['group','day']).agg({'event_name':'count'}).reset_index()



plt.figure(figsize = (23, 4))
plt.grid(True)
plt.title('Распределние количества событий по дням', size=14)

sns.barplot(data = count_day_ev, x = 'day', y = 'event_name', hue = 'group')
plt.ylabel('Количество событий')
plt.xlabel('День')
plt.xticks(rotation = 45)
plt.show()


In [ ]:
# посмотрим на маркетинговые события в период АБ теста
marketing_events = marketing_events.query('start_dt >= "2020-12-07" and finish_dt <= "2021-01-04"')
print('События маркетинговых активностей в период теста')
marketing_events

Вывод<br>

Группа A: 
- Среднее количество 2.25 событий в день. 
- Наимельшее колличество событий 2 затем 3 затем 1 затем 4 затем 6 затем 8 

- На графике распрделения событий по дням видно как c 14 по 23 декабря наблюдается пик количества событий, а после 23го пошел спад, с учетом того что была маркетингоая активность с 25 декабря по 03 января Christmas&New Year Promo.
- Рост событий может быть вызван сезонностью перед новогодними праздниками


Группа B: 
- Среднее количество 2.05 событий в день. 
- Наимельшее колличество событий 2 затем 1 затем 3 затем 4

- На графике распрделения событий по дням видно что нет вырадденно активности как c 14 по 20 декабря наблюдается пик количества событий, а после 23 го пошел спад, с учетом того что была маркетингоая активность с 25 декабря по 03 января Christmas&New Year Promo.
- Рост событий может быть вызван сезонностью перед новогодними праздниками




In [ ]:
# выделим в датафреймы каждый из тестов
test1 = participants.query('ab_test == "recommender_system_test"')
test2 = participants.query('ab_test == "interface_eu_test"')

ff = test1.merge(test2, on='user_id', how='inner')

unique_users = ff['user_id'].unique().tolist()
len(unique_users)


In [ ]:
# теперь проверим есть ли пользователи группы А в группе В и наоборот.
matching = lambda a,b: '{} совпадений'.format(len(set(a) & set(b))) if len(set(a) & set(b)) else 'Совпадений нет'

rst_group = participants[participants['ab_test'] == 'recommender_system_test']
A = rst_group[rst_group['group'] == 'A']['user_id']
B = rst_group[rst_group['group'] == 'B']['user_id']
print(matching(A,B))


In [ ]:
# посмотрим количество пользователей в каждой группе эксперимента
groupby_exp = rst_group.groupby('group').agg({'user_id':'nunique'}).reset_index()

groupby_exp['part_of_total'] = groupby_exp["user_id"] / groupby_exp["user_id"].sum()
print("Количество пользователей каждой из групп")
groupby_exp

В сумме  6701 пользователей – это соответствует ТЗ<br>
Количество пользователей группы А 57%  – это может негативно сказаться на тестировании, так как группа должна не привышать хотябы 51%

<b>Вывод по иследовательскому анализу</b> 

<b>Воронка</b> 

Вывод <br>
Группа А
- В группе А  18801 событий, 6223 пользователей
- В воронке: Переход с логина на просмотр карточки товара 65%
Переход с карточки товара в корзину 46%
Переход из корзины в покупку в корзину 106% (пользователей больше 100%, это может быть связано с переходом к покупке сразу с карточки товара)

Группа B
- В группе B  5055 событий, 1961 пользователей
- В воронке: Переход с логина на просмотр карточки товара 56%
Переход с карточки товара в корзину 49%
Переход из корзины в покупку в корзину 100% 





<b>Распределение количества событий по дням</b> 

Вывод<br>

Группа A: 
- Среднее количество 2.25 событий в день. 
- Наимельшее колличество событий 2 затем 3 затем 1 затем 4 затем 6 затем 8 

- На графике распрделения событий по дням видно как c 14 по 23 декабря наблюдается пик количества событий, а после 23го пошел спад, с учетом того что была маркетингоая активность с 25 декабря по 03 января Christmas&New Year Promo.
- Рост событий может быть вызван сезонностью перед новогодними праздниками


Группа B: 
- Среднее количество 2.05 событий в день. 
- Наимельшее колличество событий 2 затем 1 затем 3 затем 4

- На графике распрделения событий по дням видно что нет вырадденно активности как c 14 по 20 декабря наблюдается пик количества событий, а после 23 го пошел спад, с учетом того что была маркетингоая активность с 25 декабря по 03 января Christmas&New Year Promo.
- Рост событий может быть вызван сезонностью перед новогодними праздниками



### А/В тестирование

<b>Статистическая разница долей z-критерием</b>

In [ ]:
print('Всего уникальных пользователей - {:.0f}' .format(data['user_id'].nunique()))

In [ ]:
events_by_groups = data\
                    .pivot_table(index = 'event_name', columns = 'group', values = 'user_id', aggfunc = 'nunique')\
                    .sort_values('A', ascending = False)

In [ ]:
events_by_groups

При проверке гипотез о равенстве среднего будем применять метод Z-тест (z-критерий Фишера), т.к. будем проводить сравнения двух независимых выборок большого размера. Критический уровень статистической значимости примем равным 5%. Создадим функцию с применением поправки Бонферрони:

In [ ]:
def z_test(group_1, group_2, eventname, alpha): 
    purchases1 = events_by_groups.loc[eventname, group_1]
    purchases2 = events_by_groups.loc[eventname, group_2]
    n1 = events_by_groups.iloc[0, 0]
    n2 = events_by_groups.iloc[0, 1]
    p1 = purchases1 / n1    
    p2 = purchases2 / n2   
    
    print(purchases1, purchases2, n1, n2)
    difference = p1 - p2    
    p_combined = (purchases1 + purchases2) / (n1 + n2)    
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / n1 + 1 / n2))                
    distr = st.norm(0, 1)    
    p_value = (1 - distr.cdf(abs(z_value))) * 2

    bonferroni_alpha = alpha / 4
    
    print('Проверка события:', eventname)
    print('p-value: ',p_value)
   
    if (p_value < bonferroni_alpha):
        print("Отвергаем нулевую гипотезу: между конверсией групп есть значимая разница")
    else:
        print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать конверсию групп разной") 
    print('')
     

<b>Формулировка гипотез</b>

H0: Между группами А и В нет различий конверсии по событиям

H1: Между группами А и В есть различия конверсии по событиям

In [ ]:
for eventname in events_by_groups.index:
    z_test('A', 'B', eventname, 0.05)

Для события login, проверка статистической разницы долей z-критерием невозможна, так как это действие совершили все пользователи группы А и группы В, а это число соответствует максимальному размеру выборки групп.

### Общие вывод исследования

<b>Цель работы:</b>

Было нужно  провести оценку A\B-тестирования новой рекомендательной системы товаров для пользователей. Оценить изменились ли продажи после внедрения изменений или нет.

Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:

конверсии в просмотр карточек товаров — событие product_page,
просмотры корзины — product_cart,
покупки — purchase.

<b>Вывод по иследовательскому анализу</b> 

<b>Воронка</b> 

Группа А
- В группе А  18246 событий, 60874 пользователей
- В воронке: Переход с логина на просмотр карточки товара 66%
Переход с карточки товара в корзину 48%
Переход из корзины в покупку в корзину 108% (пользователей больше 100%, это может быть связано с переходом к покупке сразу с карточки товара)

Группа B
- В группе B  45751 событий, 14236 пользователей
- В воронке: Переход с логина на просмотр карточки товара 64%
Переход с карточки товара в корзину 51%
Переход из корзины в покупку в корзину 99% 

<b>Распределение количества событий по дням</b> 

Группа A: 
- Среднее количество 2.41 событий в день. 
- Наимельшее колличество событий 2 затем 3 затем 1 затем 4 затем 6 затем 8 

- На графике распрделения событий по дням видно как c 14 по 23 декабря наблюдается пик количества событий, а после 23го пошел спад, с учетом того что была маркетингоая активность с 25 декабря по 03 января Christmas&New Year Promo.
- Рост событий может быть вызван сезонностью перед новогодними праздниками


Группа B: 
- Среднее количество 2.32 событий в день. 
- Наимельшее колличество событий 2 затем 3 затем 1 затем 4 затем 6 затем 8

- На графике распрделения событий по дням видно что нет вырадденно активности как c 14 по 20 декабря наблюдается пик количества событий, а после 23 го пошел спад, с учетом того что была маркетингоая активность с 25 декабря по 03 января Christmas&New Year Promo.
- Рост событий может быть вызван сезонностью перед новогодними праздниками


<b>Целевая аудитория </b>

В тестах есть 887 пересечений между грапами А и B<br>

Для эксперимента есть 6701 пользователей – это соответствует тз (6000) <br>
Количество пользователей группы А 57%  – это может негативно сказаться на тестировании, так как группа должна не привышать хотябы 51%


Одновременно с тестом recommender_system_test проходил тест interface_eu_test;<br>
887 пользователя теста recommender_system_test также попали в тест interface_eu_test;<br>

время период проведения теста совпало с двумя маркетинговыми событиями, что могло повлиять на активность пользователей;<br>
неравномерное распределение по группам А и В;<br>

нарушена логическая последовательность событий: часть пользователей оплатили покупку минуя просмотр корзины товаров.<br>

Рекомендации: учесть выявленные недочеты и доработать механизм и методику проведения теста.<br>



<b> По проведенному А/B тесту. Статистическая разница долей z-критерием</b>

Проверка события: login<br>
p-value:  nan<br>
Не получилось отвергнуть нулевую гипотезу, нет оснований считать конверсию групп разной<br>

Проверка события: product_page<br>
p-value:  5.084<br>
Отвергаем нулевую гипотезу: между конверсией групп есть значимая разница<br>

Проверка события: purchase<br>
p-value:  0.018<br>
Не получилось отвергнуть нулевую гипотезу, нет оснований считать конверсию групп разной<br>

Проверка события: product_cart<br>
p-value:  0.150<br>
Не получилось отвергнуть нулевую гипотезу, нет оснований считать конверсию групп разной<br>



Для события login, проверка статистической разницы долей z-критерием невозможна, так как это действие совершили все пользователи группы А и группы В, а это число соответствует максимальному размеру выборки групп.<br>



